# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science: 
## Homework 5: High Dimensionality and PCA

**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner

<hr style="height:2.4pt">

In [2]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [3]:
%%html
<style>
.jp-MarkdownCell {background-color: cornsilk;}
.text_cell {background-color: cornsilk;}
</style>

In [4]:
#RUN THIS CELL
import os
import pathlib
working_dir = pathlib.Path().absolute()
os.chdir(working_dir)

<hr style="height:2pt">

### INSTRUCTIONS

- To submit your assignment follow the instructions given in Canvas.

- This homework can be submitted in pairs, and it is encouraged for you to do so. Especially during covid and distancing, this can be a way to work with other students and learn alongside one another. As future data scientists, you will often be expected to work with others, and working in pairs can help practice communicating data science concepts.

- Please restart the kernel and run the entire notebook again before you submit.

- Running cells out of order is a common pitfall in Jupyter Notebooks. To make sure your code works restart the kernel and run the whole notebook again before you submit. Exceptions should be made for code with a long execution time, of course.
- We have tried to include all the libraries you may need to do the assignment in the imports statement at the top of this notebook. We strongly suggest that you use those and not others as we may not be familiar with them. .
- Please use .head() when viewing data. Do not submit a notebook that is **excessively long**. 
- In questions that require code to answer, such as "calculate the $R^2$", do not just output the value from a cell. Write a `print()` function that includes a reference to the calculated value, **not hardcoded**. For example: 
```
print(f'The R^2 is {R:.4f}')
```

<hr style="height:2pt">

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.decomposition import PCA

<div class='theme'> Cancer Classification from Gene Expressions </div>

In this assignment, we will build several classification models to distinguish between two related classes of cancer, acute lymphoblastic leukemia (ALL) and acute myeloid leukemia (AML), using gene expression measurements. The .csv data file is provided in the compressed file `data/hw5_genes_multiclass.zip`. Each row in this file corresponds to a tumor tissue sample from a patient with one of the two forms of leukemia.  Note: there are two different forms of the response variable. 

- The first column contains `Cancer_type`: **0 = ALL** class and **1 = AML** class
- Columns 2-7130 contain expression levels of 7129 genes recorded from each tissue sample 
- The last column `Cancer_subtype` additionally distinguishes between two subtypes of ALL, subtype T and subtype B (used in problem 5): **0 = ALL subtype T**, **1 = ALL subtype B**, **2 = type AML**. 

In the following questions, we will use logistic regression and PCA to build classification models for this data set. 

## Contents
- [Question 1 [20 pts]: Data Exploration](#Question-1-[20-pts]:-Data-Exploration) 
- [Question 2 [25 pts]: Logistic Regression Modeling](#Question-2-[25-pts]:-Logistic-Regression-Modeling) 
- [Question 3 [20 pts]: Performing Principal Components Analysis](#Question-3-[20-pts]:-Performing-Principal-Components-Analysis)
- [Question 4 [10 pts]: Principal Components Regression (PCR)](#Question-4-[10-pts]:-Principal-Components-Regression-(PCR))
- [Question 5 [25 pts]: Multi-Class Response](#Question-5-[25-pts]:-Multi-Class-Response)

#### <div class='exercise'><b>Question 1 [20 pts]: Data Exploration</b></div>

[▲ Return to contents](#Contents)

The first step is to split the observations into an approximate 75-25 train-test split.  Below we provide some code to do this for you (we want to make sure everyone has the same splits). It also prints the dataset shape before splitting and after splitting. `Cancer_type` is our response variable for problems 1-4, while `Cancer_subtype` is the response variable in problem 5.


**1.1** Take a peek at your training set: you should notice the severe differences in the measurements from one gene to the next (some are negative, some hover around zero, and some are well into the thousands). To account for these differences in scale, normalize each predictor to vary between 0 and 1.   **NOTE: for the entirety of this homework assignment, you will use these normalized values, not the original, raw values**.


**1.2** The training set contains more predictors than observations. What problem(s) can this lead to in fitting a classification model to such a dataset? Explain in 3 or fewer sentences.


**1.3** Determine which 10 genes best individually discriminates between the two cancer classes (Note: consider every gene in the dataset).  Then determine the single best gene predictor.  Plot two sets of histograms of your `best_predictor` -- one using the training set and another using the test set.  The histograms should clearly distinguish two different `Cancer_type` classes.

**Hint:** You may use any reasonable approach to determine the `best_predictor`, but please use something very simple (whether taught in this class or elsewhere).

**1.4** Using `best_predictor`, create a classification model by eye-balling a value for this gene that would best discriminate the two cancer classes in the training set. (Note: Do not use an algorithm to determine for you the optimal coefficient or threshold; we are asking you to provide a rough estimate / model by manual inspection.) Justify your choice of value in 1-2 sentences. Report the accuracy of your hand-chosen model on the train and test sets.

---


In [6]:
#############################
## DO NOT MODIFY THIS CODE ##
#############################

np.random.seed(109)
#zf = zipfile.ZipFile('data/hw5_genes_multiclass.csv.zip') 
df = pd.read_csv('data/hw5_genes_multiclass.csv')
X = df.drop(['Cancer_type','Cancer_subtype'], axis=1)
X_train, X_test, y_train, y_test, y2_train, y2_test  = train_test_split(
    X, df.Cancer_type, df.Cancer_subtype, test_size=0.25, random_state = 109,
    stratify = df.Cancer_subtype)

print(df.shape)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(df.Cancer_type.value_counts(normalize=True))

(752, 7131)
(564, 7129) (188, 7129) (564,) (188,)
0.0    0.511968
1.0    0.488032
Name: Cancer_type, dtype: float64



## Answers

<div class='exercise-r'>  
 
**1.1** Take a peek at your training set: you should notice the severe differences in the measurements from one gene to the next (some are negative, some hover around zero, and some are well into the thousands). To account for these differences in scale, normalize each predictor to vary between 0 and 1.   **NOTE: for the entirety of this homework assignment, you will use these normalized values, not the original, raw values**.
 
 
 </div>

In [7]:
# your code here


<div class='exercise-r'>  
 
**1.2** The training set contains more predictors than observations. What problem(s) can this lead to in fitting a classification model to such a dataset? Explain in 3 or fewer sentences.
 
 
 </div>

*Your answer here*

<div class='exercise-r'>  
 
**1.3** Determine which 10 genes best individually discriminates between the two cancer classes (Note: consider every gene in the dataset).  Then determine the single best gene predictor.  Plot two sets of histograms of your `best_predictor` -- one using the training set and another using the test set.  The histograms should clearly distinguish two different `Cancer_type` classes.
 
 **Hint:** You may use any reasonable approach to determine the `best_predictor`, but please use something very simple (whether taught in this class or elsewhere).
 
 </div>

In [9]:
# your code here


<div class='exercise-r'>  
    
**1.4** Using `best_predictor`, create a classification model by eye-balling a value for this gene that would best discriminate the two cancer classes in the training set. (Note: Do not use an algorithm to determine for you the optimal coefficient or threshold; we are asking you to provide a rough estimate / model by manual inspection.) Justify your choice of value in 1-2 sentences. Report the accuracy of your hand-chosen model on the train and test sets.

In [11]:
# your code here


*Your answer here*

#### <div class='exercise'><b>Question 2 [25 pts]: Logistic Regression Modeling</b></div>

[▲ Return to contents](#Contents)

**2.1** Fit a simple logistic regression model to the training set using the single gene predictor `best_predictor` to predict cancer type. Calculate and display the training and test classification accuracies of this model. 
How do accuracies compare to the eye-balled ones from 1.4? 

**2.2** Next, fit a multiple logistic regression model with **all** the gene predictors from the data set (reminder: for this assignment, we are always using the normalized values).  
How does the classification accuracy of this model compare with the logistic and eyeballed models above which were fit with a single gene? Be sure to evaluate both the training and test sets.

**2.3** Print out the logistic regression coefficients for  `best_predictor` from the simple logistic and multiple logistic regression models in part 1 and part 2 above.  Interpret the coefficients: Do they agree or disagree?  What does this indicate?

**2.4** Now let's use regularization to improve the predictions from the multiple logistic regression model. Specifically, use LASSO-like regularization and 5-fold cross-validation to fit the model on the training set. 
Report the chosen best value of the regularization hyperparamter and the classification accuracy on both the training and test sets.

**2.5** Compare the classification accuracies (both train and test) between the un-regularized multiple logistic regression model to the regularized one. Briefly explain why these results occur.

**2.6** How many predictors are considered as important features in this regularized model?  What does that say about the full logistic regression model in problem 2.2?


## Answers

<div class='exercise-r'>  
 
**2.1** Fit a simple logistic regression model to the training set using the single gene predictor `best_predictor` to predict cancer type. Calculate and display the training and test classification accuracies of this model.
 How do accuracies compare to the eye-balled ones from 1.4?
 
 </div>

In [12]:
# your code here


*Your answer here*

<div class='exercise-r'>  
 
**2.2** Next, fit a multiple logistic regression model with **all** the gene predictors from the data set (reminder: for this assignment, we are always using the normalized values).
 How does the classification accuracy of this model compare with the logistic and eyeballed models above which were fit with a single gene? Be sure to evaluate both the training and test sets.
 
 </div>

In [13]:
# your code here


*Your answer here*

<div class='exercise-r'>  
 
**2.3** Print out the logistic regression coefficients for  `best_predictor` from the simple logistic and multiple logistic regression models in part 1 and part 2 above.  Interpret the coefficients: Do they agree or disagree?  What does this indicate?
 
 </div>

In [14]:
# your code here


*Your answer here*

<div class='exercise-r'>  
 
**2.4** Now let's use regularization to improve the predictions from the multiple logistic regression model. Specifically, use LASSO-like regularization and 5-fold cross-validation to fit the model on the training set.
 Report the chosen best value of the regularization hyperparamter and the classification accuracy on both the training and test sets.
 
 </div>

In [15]:
# your code here


<div class='exercise-r'>  
 
**2.5** Compare the classification accuracies (both train and test) between the un-regularized multiple logistic regression model to the regularized one. Briefly explain why these results occur.
 
 </div>

*Your answer here*

<div class='exercise-r'>  
    
**2.6** How many predictors are considered as important features in this regularized model?  What does that say about the full logistic regression model in problem 2.2?

In [17]:
# your code here


*Your answer here*

---

#### <div class='exercise'><b>Question 3 [20 pts]: Performing Principal Components Analysis</b></div>

[▲ Return to contents](#Contents)

**3.1** Create the full PCA decomposition of `X_train` and apply the transformation to both `X_train` and `X_test`.  Report the shape of both of these.  What is the limiting factor for the maximum number of PCA components for this data set? 

**3.2** PCA is often solely used to help in visualizing high-dimensional problems.  Plot the scatterplot of the second PCA vector of train on the $Y$-axis and the first PCA vector of train on the $X$-axis (be sure to denote the classes via different colors and markings).  In 2-3 sentences, explain why using the scatterplot of the top two PCA vectors is a useful approach to visualize a high dimensional classification problem.

**3.3** Determine and report the variance explained in `X_train` based on the top two PCA vectors.  Determine and report how many PCA vectors are needed so that 90\% of the variability in the predictors is explained, and create a plot to illustrate this result (Hint: look at cumulative explained variability vs. number of PCA components used).

**3.4** Plot explained variability in the predictors on the $Y$-axis and the PCA component number on the $X$-axis. Select a reasonable value for the number of components that balances representativeness (of the predictors) with parsimony.

<hr>


## Answers

<div class='exercise-r'>  
 
**3.1** Create the full PCA decomposition of `X_train` and apply the transformation to both `X_train` and `X_test`.  Report the shape of both of these.  What is the limiting factor for the maximum number of PCA components for this data set?
 
 </div>

In [18]:
# your code here


*Your answer here*

<div class='exercise-r'>  
 
**3.2** PCA is often solely used to help in visualizing high-dimensional problems.  Plot the scatterplot of the second PCA vector of train on the $Y$-axis and the first PCA vector of train on the $X$-axis (be sure to denote the classes via different colors and markings).  In 2-3 sentences, explain why using the scatterplot of the top two PCA vectors is a useful approach to visualize a high dimensional classification problem.
 
 </div>

In [19]:
# your code here


*Your answer here*

<div class='exercise-r'>  
 
**3.3** Determine and report the variance explained in `X_train` based on the top two PCA vectors.  Determine and report how many PCA vectors are needed so that 90\% of the variability in the predictors is explained, and create a plot to illustrate this result (Hint: look at cumulative explained variability vs. number of PCA components used).
 
 </div>

In [20]:
# your code here


<div class='exercise-r'>  
    
**3.4** Plot explained variability in the predictors on the $Y$-axis and the PCA component number on the $X$-axis. Select a reasonable value for the number of components that balances representativeness (of the predictors) with parsimony.

In [21]:
# your code here


*Your answer here*

#### <div class='exercise'><b>Question 4 [10 pts]: Principal Components Regression (PCR)</b></div>

[▲ Return to contents](#Contents)

**4.1** Fit three separate Logistic Regression models using principal components as the predictors: (1) with just the first two PCA vectors, (2) with the number of component vectors you chose from problem 3 above, and (3) with the number of components that explain at least 90% of the variability in the predictor set. How do the classification accuracy values on both the training and test sets compare with the models fit in Question 2?

**4.2** Use cross-validation to determine the best number of principal components. Try out the 3 values from the previous sub-part and optionally include other values as well. 


<hr>


## Answers

<div class='exercise-r'>  
 
**4.1** Fit three separate Logistic Regression models using principal components as the predictors: (1) with just the first two PCA vectors, (2) with the number of component vectors you chose from problem 3 above, and (3) with the number of components that explain at least 90% of the variability in the predictor set. How do the classification accuracy values on both the training and test sets compare with the models fit in Question 2?
 
 </div>

In [22]:
# your code here


*Your answer here*

<div class='exercise-r'>  
    
**4.2** Use cross-validation to determine the best number of principal components. Try out the 3 values from the previous sub-part and optionally include other values as well.

In [23]:
# your code here


#### <div class='exercise'><b>Question 5 [25 pts]: Multi-Class Response</b></div>

[▲ Return to contents](#Contents)

As mentioned earlier, there are actually two subtypes of `ALL` cancer: B-cell and T-cell.  The variable `Cancer_subtype` designates the 3 cancer cubtypes: 
- 0 = ALL Type T, 
- 1 = ALL Type B, and 
- 2 = AML.  
Use this updated response variable to answer the following questions:

**5.1** Fit two separate well-tuned, regularized (Ridge-like), multinomial logistic regression models to predict `Cancer_subtype`. For the first model, use the first two PCA components as predictors. For the second model, include  the quadratic and interaction terms for the first two PCA components, for a total of five predictors. Print and evaluate (with two or three sentences) the classification accuracy of the two models, on both train and test.

**5.2** Create two separate scatter plots (one will be for each model above) of the first two principal components of the test data and denote the three cancer types by different color and marker.  Plot the decision boundaries separately on the two scatterplots and interpret the results:  which model appears to draw more reasonable decision boundaries?  Do the first two principal components appear to provide enough predictive power for this classification problem?

**Hint: you can use the `meshgrid` as seen in lecture exercises to generate the decision boundaries.**

**5.3** Use cross-validation to determine the best number of principal components for this multiclass problem. Consider the set of [2, 5, 10, 15, 20, 50, and 100] components.  Be sure to clearly substantiate your choice.

**5.4** For your best model in the previous part, determine the classification accuracies within each observed (not predicted) subtype in test: which group is the most difficult to classify accurately?  Is this surprising?  Why or why not?

**5.5** In the 2-class problem the classification threshold can be altered from 0.5 to affect the number of false positives and false negatives (this is what the ROC curve is based on). In the multiclass setting, the predicted class probabilities can be weighted to determine the winning class (so that a smaller class will not be 'ignored' by the algorithm): essentially, each class's predicted probability can be multiplied by this class weight and compared to determine the classification.
 
Determine the classification accuracies in each observed subtype in test if instead the observations were weighted more 'fairly': they are inversely weighted based on the observed sample sizes in train (so that if there were just 2 groups in the response with 75% of the response in one class and 25% in the other, the smaller class should be weighted 3 times as much as the larger class).  

Report the weights you used and the resulting classification accuracies in each subtype.  In what way have the results improved?  In what way have the results not improved?  Comment with 2-3 sentences.

<hr>


## Answers

<div class='exercise-r'>  
 
**5.1** Fit two separate well-tuned, regularized (Ridge-like), multinomial logistic regression models to predict `Cancer_subtype`. For the first model, use the first two PCA components as predictors. For the second model, include  the quadratic and interaction terms for the first two PCA components, for a total of five predictors. Print and evaluate (with two or three sentences) the classification accuracy of the two models, on both train and test.
 
 </div>

In [25]:
# your code here


*Your answer here*

<div class='exercise-r'>  
 
**5.2** Create two separate scatter plots (one will be for each model above) of the first two principal components of the test data and denote the three cancer types by different color and marker.  Plot the decision boundaries separately on the two scatterplots and interpret the results:  which model appears to draw more reasonable decision boundaries?  Do the first two principal components appear to provide enough predictive power for this classification problem?
 
 **Hint: you can use the `meshgrid` as seen in lecture exercises to generate the decision boundaries.**
 
 </div>

In [26]:
# your code here


<div class='exercise-r'>  
 
**5.3** Use cross-validation to determine the best number of principal components for this multiclass problem. Consider the set of [2, 5, 10, 15, 20, 50, and 100] components.  Be sure to clearly substantiate your choice.
 
 </div>

In [27]:
# your code here


*Your answer here*

<div class='exercise-r'>  
 
**5.4** For your best model in the previous part, determine the classification accuracies within each observed (not predicted) subtype in test: which group is the most difficult to classify accurately?  Is this surprising?  Why or why not?
 
 </div>

In [29]:
# your code here


<div class='exercise-r'>  
 
**5.5** In the 2-class problem the classification threshold can be altered from 0.5 to affect the number of false positives and false negatives (this is what the ROC curve is based on). In the multiclass setting, the predicted class probabilities can be weighted to determine the winning class (so that a smaller class will not be 'ignored' by the algorithm): essentially, each class's predicted probability can be multiplied by this class weight and compared to determine the classification.
 
 Determine the classification accuracies in each observed subtype in test if instead the observations were weighted more 'fairly': they are inversely weighted based on the observed sample sizes in train (so that if there were just 2 groups in the response with 75% of the response in one class and 25% in the other, the smaller class should be weighted 3 times as much as the larger class).
 
 Report the weights you used and the resulting classification accuracies in each subtype.  In what way have the results improved?  In what way have the results not improved?  Comment with 2-3 sentences.
 </div>

In [30]:
# your code here
